## Training a Simple ViT using Pytorch [20 marks]

In this question, you will implement a vision transformer based image classification model using pytorch.
Implement a basic version of vision transformer (https://arxiv.org/pdf/2010.11929.pdf), that first divides an image into patches and then passes them through a set of multihead self attention modules to perform classification. Please check out the details in the paper. Note that classification happens from the CLS token of the final transformer layer.

[Experiment 1] Train this model on the CIFAR-10 dataset for 10-class classification. Keep the number of attention heads to be 4 for all the experiments. [6 points]

[Experiment 2] Try out different patch sizes (like 4x4, 8x8, 16x16). You can divide the image into both overlapping and non-overlapping patches. [4 points]

[Experiment 3] How does model performance change if you vary the number of attention heads? [4 points]

[Experiment 4] Perform classification by using the CLS token from different layers of the model. [6 points]
Create a detailed report of all the experiments and analyses.


In [27]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn

import numpy as np
# from patchify import patchify

from tqdm import tqdm, trange

from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader

from torchvision.transforms import ToTensor
import torch.optim as optim
from torchvision.datasets import CIFAR10
from sklearn.model_selection import train_test_split


np.random.seed(0)
torch.manual_seed(0)

In [46]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
num_classes = 10
patch_size = 4
batch_size = 64
num_epochs = 10
learning_rate = 0.001
image_size = 32  # CIFAR-10 image size
num_patches = (32 // patch_size) ** 2  # Assuming input image size is 32x32 for CIFAR-10
depth = 6  # Number of transformer layers
heads = 4  # Number of attention heads
dim = 256  # Dimension of the embeddings
mlp_dim = 512  # Dimension of the feedforward layer

In [54]:
class VisionTransformer(nn.Module):
    def __init__(self, image_size=32, patch_size=16, num_classes=10, embed_dim=256, num_heads=8, num_layers=6):
        super(VisionTransformer, self).__init__()

        self.embedding = nn.Conv2d(3, embed_dim, kernel_size=patch_size, stride=patch_size)
        self.cls_token = nn.Parameter(torch.randn(1, 1, embed_dim))

        self.transformer_blocks = nn.ModuleList([
            nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads) for _ in range(num_layers)
        ])

        self.fc = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = x.flatten(2).permute(2, 0, 1)  # Reshape for transformer input
        cls_token = self.cls_token.repeat(x.size(0), 1, 1)
        x = torch.cat((cls_token, x), dim=1)


        for transformer_block in self.transformer_blocks:
            x = transformer_block(x.permute(1, 0, 2))  # Permute for transformer input
            x = x.permute(1, 0, 2)  # Permute back to the original shape

        x = x.mean(dim=0)  # Global average pooling
        x = self.fc(x)
        return x


In [31]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [32]:
train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)

# Split into train and validation sets
train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.1, random_state=42)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)


Files already downloaded and verified
Files already downloaded and verified


In [56]:
model = VisionTransformer(32, patch_size, num_classes,256, num_heads,num_layers).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs[:labels.size(0),:], labels)
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    with torch.no_grad():
        total_correct = 0
        total_samples = 0
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total_samples += labels.size(0)
            total_correct += (predicted == labels).sum().item()

        accuracy = total_correct / total_samples
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}, Validation Accuracy: {accuracy:.4f}')


In [60]:
model.eval()
with torch.no_grad():
    total_correct = 0
    total_samples = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs[:labels.size(0),:], 1)
        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

    accuracy = total_correct / total_samples
    print(f'Test Accuracy: {accuracy:.4f}')

Test Accuracy: 0.1000


### Rough work